In [ ]:
import os, random


class Drivev4(Megadrive):
    def set(self):
        self.files = ['list.txt', 'path.txt', 'rclone.conf']

    def init(self):
        content = self.read_file('path.txt')
        if len(content) < 2:
            self.endmsg = 'Args corrupted'
            return False
        self.from_path = content[0]
        self.to_path = content[1]
        if not os.path.isfile('/usr/bin/rclone'):
            print('----- Installing rclone -----')
            !curl https://rclone.org/install.sh | sudo bash
        return True
    
    def check_phase(self):
        if random.randrange(0, 100) <= 20:
            get_ipython().system_raw(f"rclone --config=/content/rclone.conf lsf 'TO:{self.to_path}' --files-only -R --files-from /content/list.txt > file_completed")
            get_ipython().system_raw(f"comm -23 /content/list.txt file_completed --nocheck-order > file_onqueue")
            get_ipython().system_raw("wc -l < file_completed > line_completed")
            get_ipython().system_raw("wc -l < file_onqueue > line_onqueue")
            with open('line_completed', 'r') as f:
                self.file_completed = int(f.read())
            with open('line_onqueue', 'r') as f:
                self.file_total = int(f.read())
            self.log('INFO', f'{self.file_completed} files completed. {self.file_total} files on queue.')
            self.endmsg = f'Completed. {self.file_total} files transfered. {self.file_completed} files completed'
        else:
            get_ipython().system_raw('cp /content/list.txt file_onqueue')
            get_ipython().system_raw("wc -l < file_onqueue > line_onqueue")
            with open('line_onqueue', 'r') as f:
                self.file_total = int(f.read())
            self.log('INFO', f'Skip checking. {self.file_total} files on queue.')
            self.endmsg = f'{self.file_total} files transferred (no checking).'
    
    def transfer_phase(self):
        args = '--transfers=2' if self.file_total < 50 else ''
        get_ipython().system_raw(f"rclone --config=/content/rclone.conf copy 'FROM:{self.from_path}' 'TO:{self.to_path}' --files-from file_onqueue {args} --no-check-certificate --drive-acknowledge-abuse -v")
